# Introduction à Numba

<img src="figures/numba_blue_icon_rgb.png" alt="Drawing" style="width: 20%;"/>

<center>**Loic Gouarin**</center>
<center>*6 juillet 2017*</center>

Tous les supports de ce cours se trouvent à l'adresse suivante

https://github.com/gouarin/cours_numba_2017

## Qu'est-ce que Numba ?

Numba permet d'optimiser un code numérique écrit en Python en s'appuyant sur LLVM. Il fait du just-un-time, de l'inférence de types et de la compilation de fonctions.

- **compilation de fonctions**

Numba compile des fonctions Python. Il n'est pas conseillé de compiler l'ensemble d'une application mais uniquement les points chauds que l'on peut identifier en utilisant des outils de profiling. Numba est un module Python comme un autre qui vous permet de rendre une fonction plus rapide.

- **inférence de type**

Python ne fait pas de typage et est donc générique. Mais ceci a un coût: Python est obligé de vérifier à chaque opération si il peut faire celle-ci en fonction des types donnés. Numba va spécialiser votre fonction pour un certain nombre de types d'entrées ce qui va permettre d'accélérer votre code Python.  

- **just-in-time**

Numba permet de générer du code optimisé sans phase de compilation (ce qui n'est pas le cas lorsque l'on veut faire de l'ahead-of-time comme nous le verrons par la suite). La génération se fait à chaque premier appel à la fonction ou lorsque les types de données d'entrée changent.

- **orienté numérique**

Numba ne s'utilise pas sur n'importe quel problème. Il est vraiment orienté calcul numérique et fonctionne bien sur les types de bases int, float, complex. En revanche, quelques difficultés apparaissent lorsque l'on veut travailler sur des chaînes de caractères. Pour avoir les meilleures performances, il est donc recommandé de travailler avec des tableaux NumPy.


Numba fonctionne sur la plupart des OS

1. Windows 7 et suivants, 32 et 64 bits
2. macOS 10.9 et suivants, 64 bits
3. Linux, 32 et 64 bits

et sur les versions Python

1. python 2.7, 3.3-3.6
2. NumPy 1.8 et suivants

et pour différentes architectures

1. x86, x86_64/AMD64 CPUs
2. NVIDIA CUDA GPUs (Compute capability 3.0 et suivants, CUDA 7.5 et suivant)
3. AMD GPUs (expérimental)
4. ARM (expérimental)

Dans la suite, nous nous intéresserons uniquement aux versions CPUs et GPUs.

## Installation

Le plus simple pour installer Numba est de passer par Anaconda et de taper la commande suivante

```bash
conda install numba
```

Si vous voulez mettre à jour votre installation de Numba

```bash
conda update numba
```

## Les fonctions fournies

Numba s'appuie essentiellement sur trois fonctions

- **jit**

jit est la fonction centrale et permet d'optimiser n'importe quelle fonction Python.

- **vectorize**

vectorized prend des scalaires en entrée et est utilisé pour la création d'[ufuncs](https://docs.scipy.org/doc/numpy/reference/ufuncs.html) NumPy. Les ufuncs travaillent élément par élément. La plupart des ufuncs NumPy sont écrites en C. L'avantage avec Numba est que vous pouvez créer vos propres ufuncs sans passer par le langage C et créer ainsi des fonctions qui peuvent devenir assez complexes.

- **guvectorize**

guvectorize est une version étendue de vectorize. Au lieu de travailler élément par élément, vous donnez en paramètres de votre fonction des tableaux qui peuvent être des entrées ou des sorties. Les dimensions peuvent être différentes étant donné que vous n'avez plus la contrainte élément par élément.

L'appel à ces fonctions se font via des décorateurs Python.

## Qu'est-ce qu'un décorateur en Python ?

Les décorateurs en Python permettent de modifier le comportement par défaut de fonctions ou de classes décorées. C'est au décorateur de spécifier comment la fonction sera appelée.

En voici un exemple simple

In [7]:
def qui(function):
    def wrapper(*args, **kargs):
        print('*'*30)
        print('appel de la fonction {}'.format(function.__name__))
        print('*'*30)
        return function(*args, **kargs)
    return wrapper

@qui
def f(a, b):
    return a + b

print(f(1, 2))

******************************
appel de la fonction f
******************************
3


Il est possible d'enchaîner les décorateurs.

In [12]:
def timeit(function):
    def wrapper(*args, **kargs):
        import time
        t1 = time.time()
        result = function(*args, **kargs)
        t2 = time.time()
        print("execution time", t2-t1)
        return result
    return wrapper

In [14]:
@qui
@timeit
def f(a, b):
    return a + b

print(f(1, 2))

******************************
appel de la fonction wrapper
******************************
execution time 3.5762786865234375e-06
3


## Premier cas d'usage

Comme dit précédemment, Numba s'utilise àl'aide de décorateurs. Il est possible de mettre différentes options lors de l'appel aux décorateurs mais nous verrons ça dans la suite lorsque nous aborderons plus en profondeur chacune des fonctionnalités.

Voici donc un exemple simple pour utiliser Numba.

In [2]:
from numba import jit

@jit
def sum(a, b):
    return a + b

In [4]:
print(sum(1, 2))
print(sum(1j, 2))

3
(2+1j)


In [5]:
import numpy as np
x = np.random.rand(10)
y = np.random.rand(10)
print(sum(x, y))

[ 0.81123154  1.08796021  1.2178957   0.3011276   1.02005484  0.36610985
  1.06050852  1.24070727  0.91881875  1.43708172]


## Comment ça marche ?

Numba s'appuie sur LLVM. LLVM permet d'avoir un outil commun à différents langages en proposant une représentation intermédiaire. En s'appuyant sur cette représentation, LLVM utilise un outil d'optimisation permettant d'accélérer des parties du code. Il génére ensuite le code machine pour plusieurs architectures (X86, ARM, PTX, ...).

![Numba Flowchart](figures/LLVM_base.png)

Numba se sert donc de ce principe. Lorsque vous appelez la fonction à optimiser via jit, il fournit une représentation intermédiare qu'il passe à LLVM qui suit ensuite le processus décrit précédemment.

![Numba Flowchart](figures/LLVM_numba.png)

Nous allons à présent aller un peu plus profondément dans les fonctionnalités de Numba.

In [1]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()